In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
from PIL import Image


In [ ]:
def load_data(image_dir, mask_dir, calipers_dir, img_size=(176, 320)):
    images, masks, calipers = [], [], []
    
    image_files = {os.path.splitext(f)[0]: f for f in os.listdir(image_dir)}
    mask_files = {os.path.splitext(f)[0]: f for f in os.listdir(mask_dir)}
    caliper_files = {os.path.splitext(f)[0]: f for f in os.listdir(calipers_dir)}
    
    common_keys = set(image_files.keys()) & set(mask_files.keys()) & set(caliper_files.keys())
    
    for key in sorted(common_keys):
        img_path = os.path.join(image_dir, image_files[key])
        mask_path = os.path.join(mask_dir, mask_files[key])
        caliper_path = os.path.join(calipers_dir, caliper_files[key])
        
        try:
            img = Image.open(img_path).convert("L")
            mask = Image.open(mask_path).convert("L")
            img = img.resize(img_size)
            mask = mask.resize(img_size)
            
            img = np.array(img) / 255.0
            mask = np.array(mask) / 255.0
        except Exception as e:
            print(f"Error al cargar la imagen o máscara {key}: {e}")
            continue
        
        caliper = np.load(caliper_path)
        
        images.append(img)
        masks.append(mask)
        calipers.append(caliper.flatten())
    
    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)
    masks = np.array(masks).reshape(-1, img_size[0], img_size[1], 1)
    inputs = np.concatenate([images, masks], axis=-1) 
    calipers = np.array(calipers)
    
    return inputs, calipers

def create_calipers_cnn(input_shape=(176, 320, 2), learning_rate=0.001):
    inputs = keras.Input(shape=input_shape)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
    x = layers.Dropout(0.1)(x)  # Regularización
    
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.1)(x) 
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(4, activation='linear')(x) 
    
    model = keras.Model(inputs, outputs, name="calipers_cnn")
    
    return model

In [ ]:
learning_rate = 0.0005
epochs = 100
batch_size = 16

model = create_calipers_cnn(learning_rate=learning_rate)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file=r"C:\Users\Mafe Valenzuela\Pictures\Saved Pictures", show_shapes=True, show_layer_names=True, dpi=100)


In [ ]:
train_image_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\train\images"
train_mask_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\train\masks_generates"
train_calipers_dir= r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\train\callipers"

val_image_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\images"
val_mask_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\masks_generates"
val_calipers_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\callipers"


In [ ]:
train_inputs, train_calipers = load_data(train_image_dir, train_mask_dir, train_calipers_dir)
val_inputs, val_calipers = load_data(val_image_dir, val_mask_dir, val_calipers_dir)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=10,          
    restore_best_weights=True  
)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [ ]:
model.fit(train_inputs, train_calipers, epochs=epochs, batch_size=batch_size, validation_data=(val_inputs, val_calipers),callbacks=[early_stopping])

In [ ]:
model.save(r"C:\Users\Mafe Valenzuela\Documents\Cps\Códigos\calipers_cnn.keras") 